<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_Sec49AdvanceDL/blob/master/Assignment_4/Assignment_4_extra_b)_demo_various_meta_learning_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Relation Network Using Tensorflow

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
classA = np.random.rand(1000,18)

ClassB = np.random.rand(1000,18)

In [3]:
data = np.vstack([classA, ClassB])

In [4]:
label = np.vstack([np.ones((len(classA),1)),np.zeros((len(ClassB),1))])

So, our dataset will have 2000 records

In [5]:
data.shape

(2000, 18)

In [6]:
label.shape

(2000, 1)

In [ ]:
xi = tf.placeholder(tf.float32, [None, 9])
xj = tf.placeholder(tf.float32, [None, 9])

Define the placeholder for label y,

In [ ]:
y = tf.placeholder(tf.float32, [None, 1]) 

In [ ]:
def embedding_function(x):
    
    weights = tf.Variable(tf.truncated_normal([9,1]))
    bias = tf.Variable(tf.truncated_normal([1]))
    
    a = (tf.nn.xw_plus_b(x,weights,bias))
    embeddings = tf.nn.relu(a)
    
    return embeddings

Compute the embeddings of our support set i.e $f_{\varphi}(x_i) $

In [ ]:
f_xi = embedding_function(xi)

Compute the embeddings of our query set i.e $f_{\varphi}(x_j)$

In [ ]:
f_xj = embedding_function(xj)

Now that we calculated the embeddings and have the feature vectors, we combine both the support set and query set feature vectors i.e $Z(f_{\varphi}(x_i), f_{\varphi}(x_j))$

In [ ]:
Z = tf.concat([f_xi,f_xj],axis=1)

We define our relation function $g_{\phi}()$ as three layered neural network with relu activations, 

In [ ]:
def relation_function(x):
    w1 = tf.Variable(tf.truncated_normal([2,3]))
    b1 = tf.Variable(tf.truncated_normal([3]))
    
    w2 = tf.Variable(tf.truncated_normal([3,5]))
    b2 = tf.Variable(tf.truncated_normal([5]))
    
    w3 = tf.Variable(tf.truncated_normal([5,1]))
    b3 = tf.Variable(tf.truncated_normal([1]))
    
    #layer1
    z1 = (tf.nn.xw_plus_b(x,w1,b1))
    a1 = tf.nn.relu(z1)
    
    #layer2
    z2 = tf.nn.xw_plus_b(a1,w2,b2)
    a2 = tf.nn.relu(z2)
    
    #layer3
    z3 = tf.nn.xw_plus_b(z2,w3,b3)

    #output
    y = tf.nn.sigmoid(z3)
    
    return y

We now pass the concatenated feature vectors of support set and query set to the relation function and get the relation scores, 

In [ ]:
relation_scores = relation_function(Z)

We define our loss function as mean squared error i.e squared difference between relation scores and actual y value. 

In [ ]:
loss_function = tf.reduce_mean(tf.squared_difference(relation_scores,y))

We can minimize the loss using adam optimizer,

In [ ]:
optimizer = tf.train.AdamOptimizer(0.1)
train = optimizer.minimize(loss_function)

Now, let's start our tensorflow session

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Now, we randomly sample data points for our support set $x_i$ and query set $x_j$ and train the network, 

In [ ]:
for episode in range(1000):
    _, loss_value = sess.run([train, loss_function], 
                             feed_dict={xi:data[:,0:9]+np.random.randn(*np.shape(data[:,0:9]))*0.05,
                                        xj:data[:,9:]+np.random.randn(*np.shape(data[:,9:]))*0.05,
                                        y:label})
    if episode % 100 == 0:
        print("Episode {}: loss {:.3f} ".format(episode, loss_value))